In [4]:
import polars as pl

In [4]:
print("Monkey patch applied. Loading Relik...")

# Now load the model
relik_model = Relik.from_pretrained(
    "sapienzanlp/relik-entity-linking-small", 
    device="cuda:0"
)

print("Relik loaded successfully!")

Monkey patch applied. Loading Relik...
                ___              __         
               /\_ \      __    /\ \        
 _ __     __   \//\ \    /\_\   \ \ \/'\    
/\`'__\ /'__`\   \ \ \   \/\ \   \ \ , <    
\ \ \/ /\  __/    \_\ \_  \ \ \   \ \ \\`\  
 \ \_\ \ \____\   /\____\  \ \_\   \ \_\ \_\
  \/_/  \/____/   \/____/   \/_/    \/_/\/_/
                                            
                                            



/home/picatto/ascii/gen-ai/.pixi/envs/pipeline/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2026-01-20 03:31:46,728] [INFO] [relik.inference.annotator.from_pretrained:700] [PID:855834] [RANK:0] Loading Relik from sapienzanlp/relik-entity-linking-small
[2026-01-20 03:31:46,732] [INFO] [relik.inference.annotator.from_pretrained:701] [PID:855834] [RANK:0] {
    '_target_': 'relik.inference.annotator.Relik',
    'index': {
        'span': {
            '_target_': 'relik.retriever.indexers.inmemory.InMemoryDocumentIndex.from_pretrained',
            'name_or_path': 'riccorl/retriever-relik-e5-small-entity-linking-aida-wikipedia-index',
        },
    },
    'metadata_fields': [],
    'reader': {
        '_target_': 'relik.reader.pytorch_modules.span.RelikReaderForSpanExtraction',
        'transformer_model': 'sapienzanlp/relik-reader-deberta-v3-base-aida',
        'use_nme': True,
    },
    'retriever': {
        'span': {
            '_target_': 'relik.retriever.pytorch_modules.model.GoldenRetriever',
            'question_encoder': 'riccorl/retriever-relik-e5-small-entity-lin

InstantiationException: Error in call to target 'relik.retriever.pytorch_modules.model.GoldenRetriever':
OutOfMemoryError('CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 44.39 GiB of which 16.06 MiB is free. Process 2559898 has 43.90 GiB memory in use. Including non-PyTorch memory, this process has 476.00 MiB memory in use. Of the allocated memory 47.73 MiB is allocated by PyTorch, and 2.27 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)')

In [3]:
def _skip_init(*args, **kwargs):
    pass

transformers.modeling_utils.PreTrainedModel._init_added_embeddings_weights_with_mean = _skip_init

In [8]:
import polars as pl
df = pl.read_parquet("/data/raid5/data/picatto/ascii/news/chunks_coref/**/*.parquet", hive_partitioning=True)
df.group_by(["month", "day"]).len()

month,day,len
i64,str,u32
11,"""29""",6183
11,"""2""",30282
11,"""25""",11854
11,"""20""",9386
11,"""13""",133176
…,…,…
11,"""14""",70134
11,"""3""",69127
11,"""10""",69443


In [56]:
df2 = pl.scan_parquet("/data/raid5/data/picatto/ascii/news/clean/**/*.parquet", hive_partitioning=True)
df2.select(['doc_id', 'day', 'year', 'month']).head().collect()

doc_id,day,year,month
str,str,i64,i64
"""779da8c5666b17ff""","""2""",2025,10
"""f7545efa311ea4a7""","""2""",2025,10
"""142340b530051524""","""2""",2025,10
"""f0732b1b348a345e""","""2""",2025,10
"""b960a468d590ed42""","""2""",2025,10


In [57]:
df_correct = df.select(['chunk_id', 'doc_id', 'chunk_index', 'chunk_text', 'chunk_word_count', 'uri', 'host', 'http_date']).join(df2.select(['doc_id', 'day', 'year', 'month']), on='doc_id').collect()
df_correct.head()

chunk_id,doc_id,chunk_index,chunk_text,chunk_word_count,uri,host,http_date,day,year,month
str,str,i64,str,i64,str,str,str,str,i64,i64
"""f004b8fe6e04566e""","""d9b5bc784ff38868""",0,"""Editor's Note: Hey readers, us…",414,"""https://www.lakeshorepublicmed…","""org.lakeshorepublicmedia""","""2025-11-13 13:07:56+00:00""","""13""",2025,11
"""b6cb30204872cdce""","""d9b5bc784ff38868""",1,"""health advocate in Nigeria,"" a…",511,"""https://www.lakeshorepublicmed…","""org.lakeshorepublicmedia""","""2025-11-13 13:07:56+00:00""","""13""",2025,11
"""e3f445a0c9275dd3""","""d9b5bc784ff38868""",2,"""now lives in Switzerland and i…",511,"""https://www.lakeshorepublicmed…","""org.lakeshorepublicmedia""","""2025-11-13 13:07:56+00:00""","""13""",2025,11
"""3031854970597cf0""","""d9b5bc784ff38868""",3,"""who influences their friend, w…",487,"""https://www.lakeshorepublicmed…","""org.lakeshorepublicmedia""","""2025-11-13 13:07:56+00:00""","""13""",2025,11
"""caa30e01c1eadf37""","""d9b5bc784ff38868""",4,"""— that final act of human kind…",327,"""https://www.lakeshorepublicmed…","""org.lakeshorepublicmedia""","""2025-11-13 13:07:56+00:00""","""13""",2025,11


In [59]:
df_correct.select('day').unique()

day
str
"""27"""
"""12"""
"""26"""
"""10"""
"""5"""
…
"""2"""
"""29"""
"""9"""


In [49]:
df.filter(pl.col('doc_id').is_in(['5101395490dc665c', 'efc9c61c27e8724a', '0359bdeb75cd5b66'])).head().collect().row(2)[3]

"AERGO to Cuban Convertible Peso Conversion Table\nAERGO to CUC Conversion Table\n- 1 AERGO0.07 CUC\n- 2 AERGO0.14 CUC\n- 3 AERGO0.22 CUC\n- 4 AERGO0.29 CUC\n- 5 AERGO0.36 CUC\n- 6 AERGO0.43 CUC\n- 7 AERGO0.51 CUC\n- 8 AERGO0.58 CUC\n- 9 AERGO0.65 CUC\n- 10 AERGO0.72 CUC\n- 50 AERGO3.61 CUC\n- 100 AERGO7.22 CUC\n- 1,000 AERGO72.17 CUC\n- 5,000 AERGO360.83 CUC\n- 10,000 AERGO721.66 CUC\nThe table above displays real-time AERGO to Cuban Convertible Peso (AERGO to CUC) conversions across a range of values, from 1 AERGO to 10,000 AERGO. It provides a quick reference for commonly checked AERGO amounts using the latest CUC market rates. This is useful for estimating values from small transactions to large holdings. If you are looking for custom AERGO to CUC amounts, please use the tool converter above.\nCUC to AERGO Conversion Table\n- 1 CUC13.85 AERGO\n- 2 CUC27.71 AERGO\n- 3 CUC41.57 AERGO\n- 4 CUC55.42 AERGO\n- 5 CUC69.28 AERGO\n- 6 CUC83.14 AERGO\n- 7 CUC96.99 AERGO\n- 8 CUC110.8 AERGO\n

In [1]:
import json
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple, Iterator, Set
from dataclasses import dataclass, field
from datetime import datetime
from tqdm import tqdm
from collections import defaultdict
from pyspark.sql import functions as F
import os
import logging
from pyspark.sql import SparkSession

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
from relik import Relik

/home/picatto/ascii/gen-ai/.pixi/envs/pipeline/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import polars as pl
df = pl.scan_parquet("/data/raid5/data/picatto/ascii/news/chunks_coref/**/*.parquet", hive_partitioning=True)
df.select(['chunk_id', 'doc_id', 'chunk_index', 'chunk_text', 'chunk_word_count', 'uri', 'host', 'http_date']).head().collect()

chunk_id,doc_id,chunk_index,chunk_text,chunk_word_count,uri,host,http_date
str,str,i64,str,i64,str,str,str
"""0cc995ac8ccf14a4""","""5101395490dc665c""",0,"""Deadpool and Wolverine directo…",479,"""https://www.gamesradar.com/ent…","""com.gamesradar""","""2025-11-13 00:42:42+00:00"""
"""6aade9ec11892ce7""","""5101395490dc665c""",1,"""31. Until then, check out News…",95,"""https://www.gamesradar.com/ent…","""com.gamesradar""","""2025-11-13 00:42:42+00:00"""
"""5141f485d598f8b0""","""efc9c61c27e8724a""",0,"""The railway connection between…",388,"""https://rusbiletnapoezd.ru/en/…","""ru.rusbiletnapoezd""","""2025-11-13 00:42:43+00:00"""
"""73b2d57e35a95150""","""0359bdeb75cd5b66""",0,"""AERGO to Cuban Convertible Pes…",497,"""https://www.mexc.fm/en-TR/pric…","""fm.mexc""","""2025-11-13 00:42:44+00:00"""
"""5c6a423fa9030966""","""0359bdeb75cd5b66""",1,"""0.36 CUC and 10 AERGO is value…",505,"""https://www.mexc.fm/en-TR/pric…","""fm.mexc""","""2025-11-13 00:42:44+00:00"""


In [ ]:
relik_model = Relik.from_pretrained("sapienzanlp/relik-entity-linking-small")
text = df.first().collect().row(0)[3]
text

'Deadpool and Wolverine director says the long running Netflix streaming show series finale is "10 out of 10 perfect": "Director Shawn levy have to say, the series finale of Stranger Things wrecked Director Shawn levy"\nDirector Shawn levy calls the the long running Netflix streaming show series finale a "masterpiece" that is "so deeply satisfying and moving"\nThere almost couldn\'t be more pressure on the series finale of the long running Netflix streaming show to deliver on the expectations of fans of the long running Netflix streaming show. Fortunately, Deadpool and Wolverine director Shawn Levy, who helmed some episodes of the long running Netflix streaming show season 5, has big praise for the series finale of Stranger Things, calling the series finale of Stranger Things a "masterpiece" and a "perfect 10 out of 10".\n"Having watched the final version of the final episode of the final season of the long running Netflix streaming show, the final version of the final episode of the f

In [ ]:
relik_model(text)

TypeError: 'NoneType' object cannot be interpreted as an integer

In [ ]:
def process_relik_output(relik_response) -> Dict[str, Any]:
    """Convert Relik object to JSON-serializable dict"""
    entities = []
    relations = []
    
    # Extract Entities
    if hasattr(relik_response, 'spans'):
        for span in relik_response.spans:
            ent = {
                "text": span.text,
                "label": getattr(span, 'label', 'ENTITY'),
                "start": span.start,
                "end": span.end
            }
            # Optional Wikipedia ID
            if hasattr(span, 'id') and span.id:
                ent['wikipedia_id'] = span.id
            entities.append(ent)
            
    # Extract Relations (Triplets)
    if hasattr(relik_response, 'triplets'):
        for triplet in relik_response.triplets:
            conf = getattr(triplet, 'confidence', 1.0)
            if conf >= CONFIDENCE_THRESHOLD:
                relations.append({
                    "head": triplet.subject.text,
                    "relation": triplet.label,
                    "tail": triplet.object.text,
                    "confidence": conf
                })
                
    return {"entities": entities, "relations": relations}
relik_model = Relik(model_name="relik/relik-large", device="cuda:0")
def gen_relik(text:str) -> Dict[str, Any]:    
    relik_res = relik_model(text)
    
    return relik_res

26/01/19 20:53:15 WARN Utils: Your hostname, utf resolves to a loopback address: 127.0.1.1; using 172.24.0.1 instead (on interface br-df65483b261a)
26/01/19 20:53:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/picatto/.ivy2/cache
The jars for the packages stored in: /home/picatto/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.spark#spark-hadoop-cloud_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-10fe3383-9382-42a5-907b-5e9d88d2e737;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/picatto/ascii/gen-ai/.pixi/envs/pipeline/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.postgresql#postgresql;42.7.0 in central
	found org.checkerframework#checker-qual;3.31.0 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.spark#spark-hadoop-cloud_2.12;3.5.0 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found com.google.cloud.bigdataoss#gcs-connector;hadoop3-2.2.14 in central
	found joda-time#joda-time;2.12.5 in central
	found com.fasterxml.jackson.core#jackson-

In [ ]:
INPUT_PATH = "/data/raid5/data/picatto/ascii/news/clean/"
OUTPUT_PATH = "/data/raid5/data/picatto/ascii/news/processed_graph/"

# Filter: Set to None to process all
TARGET_YEARS = [2024]
TARGET_MONTHS = [10]

# Relik Config
RELIK_MODEL_PATH = "sapienzanlp/relik-relation-extraction-sre-small" # Example checkpoint
CONFIDENCE_THRESHOLD = 0.5
DEVICE = "cuda:0"
BATCH_SIZE = 100

relik_model = Relik.from_pretrained(RELIK_MODEL_PATH, device=DEVICE)


In [ ]:
relik_inputs = [resolved_texts_map[idx] for idx in valid_indices]
relik_preds = [None] * len(texts)
                
if relik_inputs:
    try:
                        # Batch inference with Relik
                        # Note: Relik might not return a list if batch=1, handle carefully
        batch_relik = relik_model(relik_inputs)
                        
                        # If single output, wrap in list
        if not isinstance(batch_relik, list):
            batch_relik = [batch_relik]
                            
        for j, res in zip(valid_indices, batch_relik):
            relik_preds[j] = res
    except Exception as e:
        print(f"Relik Batch Failed: {e}")

                # --- E. Chunking & Result Construction ---
    for j in range(len(texts)):
        if j not in resolved_texts_map: continue
                    
        final_text = resolved_texts_map[j]
        relik_res = relik_preds[j]
                    
                    # Process Relik result into JSON
        graph_data = {"entities": [], "relations": []}
        if relik_res:
            graph_data = process_relik_output(relik_res)
                    
                    # Chunking
        chunks = chunk_text(final_text)
                    
                    # Create Output Rows
        base_meta = batch_meta[j]
        did = doc_ids[j]
                    
        for ch in chunks:
            cid = hashlib.sha256(f"{did}_{ch['chunk_index']}".encode()).hexdigest()[:16]
                        
            all_file_rows.append({
                'chunk_id': cid,
                'doc_id': did,
                            # Core chunk data
                'chunk_index': ch['chunk_index'],
                'chunk_text': ch['chunk_text'],
                'chunk_word_count': ch['chunk_word_count'],
                            # Graph Data (Stored as JSON string for compatibility)
                'entities_json': json.dumps(graph_data['entities']),
                'relations_json': json.dumps(graph_data['relations']),
                            # Metadata
                **base_meta
            })